In [ ]:
import pandas as pd
import glob
from src.main.rl.evaluation.phase2_evaluation import create_evaluation_df_phase2

path_to_save = "src/main/rl/evaluation/output/phase2_evaluation_results.csv"
pd.options.display.max_colwidth = 300
df = pd.DataFrame()
try:
    df = pd.read_csv(path_to_save)
except:
    pass
if df.empty:
    all_files = []
    for file in glob.glob("src/main/rl/models/*/*/*/*.zip", recursive=True):
        all_files.append(file)
    df = create_evaluation_df_phase2(path_to_save, all_files)
df

In [ ]:
df_wo_automation = df.query("automation_wrapper.isna() == True")
df_w_automation = df[df["automation_wrapper"] == "NPPAutomationWrapper"]

In [ ]:
from scipy.stats import iqr

statistics_wo = (
    df_wo_automation.set_index("full_path")
    .groupby("combination")
    .agg(
        return_mean=("cum_reward", "mean"),
        return_max=("cum_reward", "max"),
        return_std=("cum_reward", "std"),
        return_iqr=("cum_reward", iqr),
        timesteps_min=("total_timesteps", "min"),
        criticality_score_max=("criticality_score", "max"),
        criticality_score_std=("criticality_score", "std"),
        criticality_score__iqr=("criticality_score", iqr),
        scenario=("scenario", "first"),
        alg=("alg", "first"),
        action_wrapper=("action_wrapper", "first"),
        obs_wrapper=("obs_wrapper", "first"),
        automation_wrapper=("automation_wrapper", "first"),
    )
)

statistics_w = (
    df_w_automation.set_index("full_path")
    .groupby("combination")
    .agg(
        return_mean=("cum_reward", "mean"),
        return_max=("cum_reward", "max"),
        return_std=("cum_reward", "std"),
        return_iqr=("cum_reward", iqr),
        timesteps_min=("total_timesteps", "min"),
        criticality_score_max=("criticality_score", "max"),
        criticality_score_std=("criticality_score", "std"),
        criticality_score__iqr=("criticality_score", iqr),
        scenario=("scenario", "first"),
        alg=("alg", "first"),
        action_wrapper=("action_wrapper", "first"),
        obs_wrapper=("obs_wrapper", "first"),
        automation_wrapper=("automation_wrapper", "first"),
    )
)
assert len(statistics_wo) == 120
assert len(statistics_w) == 120

statistics_wo.merge(statistics_w, how="outer").sort_values("return_mean")

In [ ]:
from src.main.rl.evaluation.plots.phase2_plots import create_multi_object_plot, create_phase_2_counts_plots

figures = create_multi_object_plot(statistics_wo.merge(statistics_w, how="outer"))
for idx, fig in enumerate(figures):
    fig.savefig(
        f"src/main/rl/evaluation/plot_results/phase2_summary_plots{idx}.png",
        format="png",
        dpi=300,
    )

In [ ]:
# Whole df
for item in ["obs_wrapper", "scenario", "action_wrapper", "alg"]:
    df_special = (
        df.groupby(item)
        .agg(
            return_mean=("cum_reward", "mean"),
            return_max=("cum_reward", "max"),
            return_std=("cum_reward", "std"),
            return_iqr=("cum_reward", iqr),
        )
        .round(2)
    )
    print(df_special)
   # print(df_special.to_latex())

In [ ]:
# WO Autmation
for item in ["obs_wrapper", "scenario", "action_wrapper", "alg"]:
    df_special = (
        df_wo_automation.groupby(item)
        .agg(
            return_mean=("cum_reward", "mean"),
            return_max=("cum_reward", "max"),
            return_std=("cum_reward", "std"),
            return_iqr=("cum_reward", iqr),
        )
        .round(2)
    )
    print(df_special)

In [ ]:
#W Automation
for item in ["obs_wrapper", "scenario", "action_wrapper", "alg"]:
    df_special = (
        df_w_automation.groupby(item)
        .agg(
            return_mean=("cum_reward", "mean"),
            return_max=("cum_reward", "max"),
            return_std=("cum_reward", "std"),
            return_iqr=("cum_reward", iqr),
        )
        .round(2)
    )
    print(df_special)

In [ ]:
from scipy.stats import ttest_ind

scen1=df.query("scenario=='scenario1'")["cum_reward"]
scen2=df.query("scenario=='scenario2'")["cum_reward"]
scen3=df.query("scenario=='scenario3'")["cum_reward"]
print(ttest_ind(scen1, scen3,equal_var=False))
print(ttest_ind(scen1, scen2,equal_var=False))
print(ttest_ind(scen3, scen2,equal_var=False))

In [ ]:
act1=df.query("action_wrapper=='ActionSpaceOption1Wrapper'")["cum_reward"]
act2=df.query("action_wrapper=='ActionSpaceOption2Wrapper'")["cum_reward"]
act3=df.query("action_wrapper=='ActionSpaceOption3Wrapper'")["cum_reward"]
print(ttest_ind(act1, act2,equal_var=False))
print(ttest_ind(act1, act3,equal_var=False))
print(ttest_ind(act2, act3,equal_var=False))


In [ ]:
# TODO Use Anova
act1=df.query("alg=='ActionSpaceOption1Wrapper'")["cum_reward"]
act2=df.query("alg=='ActionSpaceOption2Wrapper'")["cum_reward"]
act3=df.query("alg=='ActionSpaceOption3Wrapper'")["cum_reward"]
act4=df.query("alg=='ActionSpaceOption3Wrapper'")["cum_reward"]
print(ttest_ind(act1, act2,equal_var=False))
print(ttest_ind(act1, act3,equal_var=False))
print(ttest_ind(act2, act3,equal_var=False))

In [ ]:
df.action_wrapper.sort_values()

In [ ]:
paths_that_fulfil_condition_wo_automation = statistics_wo.query(
    "return_max>200 and return_std<15 and timesteps_min==250 "
)
len(paths_that_fulfil_condition_wo_automation)

In [ ]:
paths_that_fulfil_condition_w_automation = statistics_w.query("return_max>200 and return_std<15 and timesteps_min==250")
len(paths_that_fulfil_condition_w_automation)

In [ ]:
from src.main.rl.evaluation.plots.phase2_plots import create_multi_object_plot, create_phase_2_counts_plots

fig = create_phase_2_counts_plots(
    paths_that_fulfil_condition_wo_automation.merge(paths_that_fulfil_condition_w_automation, how="outer")
)
fig.savefig(
    f"src/main/rl/evaluation/plot_results/phase2_count_plots.png",
    format="png",
    dpi=300,
)

In [ ]:
highest_return_wo_automation = df_wo_automation.query("cum_reward == cum_reward.max()")
print(highest_return_wo_automation[["cum_reward","combination"]])
highest_return_w_automation = df_w_automation.query("cum_reward == cum_reward.max()")
print(highest_return_w_automation[["cum_reward","combination"]])

In [ ]:
cols_to_count = ["alg", "scenario", "action_wrapper", "obs_wrapper", "automation_wrapper"]
statistics_wo_value_counts = pd.Series()
statistics_w_value_counts = pd.Series()
for col in cols_to_count:
    statistics_wo_value_counts = pd.concat(
        [statistics_wo_value_counts, paths_that_fulfil_condition_wo_automation[col].value_counts()]
    )
    statistics_w_value_counts = pd.concat(
        [statistics_w_value_counts, paths_that_fulfil_condition_w_automation[col].value_counts()]
    )
print(statistics_wo_value_counts)
print(statistics_w_value_counts)

In [ ]:
wo_automation_scenario1_action_space3 = paths_that_fulfil_condition_wo_automation.query(
    "scenario=='scenario1' and action_wrapper=='ActionSpaceOption3Wrapper' and timesteps_min==250"
)
wo_automation_scenario2_action_space3 = paths_that_fulfil_condition_wo_automation.query(
    "scenario=='scenario2' and action_wrapper=='ActionSpaceOption3Wrapper' and timesteps_min==250"
)
wo_automation_scenario3_action_space3 = paths_that_fulfil_condition_wo_automation.query(
    "scenario=='scenario3' and action_wrapper=='ActionSpaceOption3Wrapper' and timesteps_min==250"
)
w_automation_scenario1_action_space3 = paths_that_fulfil_condition_w_automation.query(
    "scenario=='scenario1' and action_wrapper=='ActionSpaceOption3Wrapper' and timesteps_min==250"
)
w_automation_scenario2_action_space3 = paths_that_fulfil_condition_w_automation.query(
    "scenario=='scenario2' and action_wrapper=='ActionSpaceOption3Wrapper' and timesteps_min==250"
)
w_automation_scenario3_action_space3 = paths_that_fulfil_condition_w_automation.query(
    "scenario=='scenario3' and action_wrapper=='ActionSpaceOption3Wrapper' and timesteps_min==250"
)
print(
    f"Only ActionSpace3 and scenario1 without automation:\n {wo_automation_scenario1_action_space3[['return_max', 'return_mean']]}"
)
print(
    f"Only ActionSpace3 and scenario2 without automation:\n {wo_automation_scenario2_action_space3[['return_max', 'return_mean']]}"
)
print(
    f"Only ActionSpace3 and scenario3 without automation:\n {wo_automation_scenario3_action_space3[['return_max', 'return_mean']]}"
)
print(
    f"Only ActionSpace3 and scenario1 with automation:\n {w_automation_scenario1_action_space3[['return_max', 'return_mean', ]]}"
)
print(
    f"Only ActionSpace3 and scenario2 with automation:\n {w_automation_scenario2_action_space3[['return_max', 'return_mean', ]]}"
)
print(
    f"Only ActionSpace3 and scenario3 with automation:\n {w_automation_scenario3_action_space3[['return_max', 'return_mean', ]]}"
)